In this step we are filtering the tweets of the Key Opinion Leaders, that we retrived in the first step. 


In [1]:
from twitter_crawler import TwitterCrawler

my_token = 'AAAAAAAAAAAAAAAAAAAAADRFOwEAAAAAaTp%2Bdd1OhobYMYb5ExPXm7IL6RA%3DDHknH402gOXoegUGxNtpC6giIjdackRLtdRx6tjrnnLeFN1ntT'
my_twitter_crawler = TwitterCrawler(bearer_token= my_token)


In [140]:
%%time
### inputs for the filter function
import os 
dir_with_all_tweets = os.path.join("KOL tweets")

score_for_KOP = 5
score_for_key_event = 5
score_for_key_words = 5

key_words = ["brexit", "eu", "deal", "Transition period", "Free trade","Trade", "Agreement", "Free trade agreement",
             "Governance", "Tariff", "Level playing field", "EU state aid rules", "Withdrawal","No deal","No-deal",
             "WTO rules", "Backstop", "Brexiteer", "Brexiter","Brextremist","Brexshit","Brextension","Chequers plan",
             "Custom union","Divorce bill","Flextension","Exit day","Hard border","Implementation period","Indicative vote",
             "Lexit","Lexiter","Meaningful vote",'Norway model',"Norway-plus model","Political declaration",
             "People's Vote","Remainer","Remoaner","Singapore-on-Thames","Trade","#Leave","Economy","Immigration",
             "Sovereighty","Voteleave","#remain","#GTTO","#vote","#thersamy","#stopbrexit","#PMQs","#backboris",
             "#PeoplesGovernment","#TakeBackControl ","#BrexitDividend","Barriers","#europeanunion"]

threshold_score = 10

KOP_excel_name = "KOP brexit.xlsx"
key_events_excel_name = "Brexit_key_events.xlsx"
stop_words_to_add = ["https"]
#########################################################
tweets_table_filtered, tweets_table, csv_files_evaluated, bigrams_models = my_twitter_crawler.filter_tweets_Brexit(
    dir_with_all_tweets = dir_with_all_tweets,
    score_for_KOP = score_for_KOP,
    score_for_key_event = score_for_key_event,
    score_for_key_words = score_for_key_words,
    key_words = key_words,
    threshold_score = threshold_score,
    KOP_excel_name = KOP_excel_name,
    key_events_excel_name = key_events_excel_name,
    stop_words_to_add = stop_words_to_add,
    stop_words_file_name = "stopwords.txt",
    verbose = True)

Step 1: Reading all the csv files
Step 2: Reading The Events Excel File
Step 3: Reading and preprocessing Key Opinion Leaders data
Step 4: Reading stop words data and adding your stop words

Step 5: Preprocess the tweets table for scoring:
Removing Stop words        | bigram, trigram, forthgram | merging the tweets table with the KOP and Events tables | 
found 14506 bigrams
found 12916 trigram
found 10896 forthgram

Finish preprocessing the tweets table, it took: 16.351 seconds
Step 6: Scoring the Tweets

Finish!
Total time: 81.85 Seconds ( 1.364 Minutes)
CPU times: user 17.3 s, sys: 1.52 s, total: 18.8 s
Wall time: 1min 22s


In [141]:
print(f"Total number of tweets of the KOL={tweets_table.shape[0]}")
print(f"Total number of filtered tweets of the KOL={ tweets_table.shape[0] - tweets_table_filtered.shape[0]}")
print(f"Total number of remained tweets of the KOL={tweets_table_filtered.shape[0]}")

Total number of tweets of the KOL=28965
Total number of filtered tweets of the KOL=26038
Total number of remained tweets of the KOL=2927


In [142]:
tweets_table_filtered.to_excel('tweets_table_filtered.xlsx')

-------------

## Tweets_table - Filtering 

In [143]:
import pandas as pd

data = pd.read_excel("tweets_table.xlsx")

### 1. Keeping only tweets which have at least one reply:

In [85]:
data.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'author_id', 'author_id_new',
       'conv_id_new', 'conversation_id', 'created_at', 'entities.mentions',
       'id', 'id_new', 'public_metrics.like_count',
       'public_metrics.quote_count', 'public_metrics.reply_count',
       'public_metrics.retweet_count', 'referenced_tweets', 'text',
       'users.created_at', 'users.description', 'users.id', 'users.name',
       'users.public_metrics.followers_count',
       'users.public_metrics.following_count',
       'users.public_metrics.listed_count', 'users.public_metrics.tweet_count',
       'users.username', 'users.verified', 'text_tokens', 'created_at_date',
       'is_in_special_date', 'Event_Date', 'Event Description',
       'Key opnion leaders of event', 'KOP_num', 'Index', 'Name', 'Born_in',
       'twitter_user_name', 'Role', 'Place', 'clean_user_name',
       'score_key_words', 'score_KOP', 'total_score'],
      dtype='object')

In [144]:
pd.DataFrame(data['public_metrics.reply_count']).sort_values(by=['public_metrics.reply_count'])

,public_metrics.reply_count
149857,0
143059,0
143060,0
275857,0
143064,0
...,...
44981,11569
256775,11588
77847,12503
298358,13541


In [145]:
data_with_replys = data[data['public_metrics.reply_count'] > 0]
pd.DataFrame(data_with_replys['public_metrics.reply_count'].sort_values())

,public_metrics.reply_count
137289,1
275156,1
275164,1
179202,1
132461,1
...,...
44981,11569
256775,11588
77847,12503
298358,13541


In [146]:
comparison = [
    ['rows_all_data', data.shape[0]],
    ['rows_data_with_replys', data_with_replys.shape[0]],
    ['Diff', (data.shape[0] - data_with_replys.shape[0])]]

from tabulate import tabulate
print(tabulate(comparison, tablefmt="grid"))

+-----------------------+--------+
| rows_all_data         | 299715 |
+-----------------------+--------+
| rows_data_with_replys | 135107 |
+-----------------------+--------+
| Diff                  | 164608 |
+-----------------------+--------+


### 2. Using key words:

In [147]:
data2 = pd.read_excel("tweets_table_filtered.xlsx")
data2

,Unnamed: 0,Unnamed: 0.1,author_id,author_id_new,conv_id_new,conversation_id,created_at,entities.mentions,id,id_new,...,Index,Name,Born_in,twitter_user_name,Role,Place,clean_user_name,score_key_words,score_KOP,total_score
0,26598,188,2797521996,author_id: 2797521996,conv_id: 740159086667411456,740159086667411456,2016-06-07T12:30:52.000Z,NaN,740159086667411456,id: 740159086667411456,...,1,David DAVIS,England,@DavidDavisMP,Secretary of State for Exiting the,European Union UK Government,DavidDavisMP,4,1,25
1,24792,451,155507136,author_id: 155507136,conv_id: 953958453554401283,953958453554401280,2018-01-18T11:53:13.000Z,"[{'start': 28, 'end': 41, 'username': 'GOettin...",953958453554401280,id: 953958453554401283,...,1,Mark RUTTE,Netherlands,@MinPres,Dutch Government,Prime Minister,MinPres,2,1,20
2,3894,26,3131144855,author_id: 3131144855,conv_id: 1069285577869205506,1069285577869206016,2018-12-02T17:47:45.000Z,NaN,1069287014867054976,id: 1069287014867054595,...,1,Boris JOHNSON,England,@BorisJohnson,Foreign Secretary,UK Government,BorisJohnson,3,1,20
3,7496,22,540274837,author_id: 540274837,conv_id: 821274533080551424,821274533080551424,2017-01-17T08:39:25.000Z,NaN,821275721310044160,id: 821275721310044160,...,1,Frances O'GRADY,England,@FrancesOGrady,General Secretary,Trades Union Congress,FrancesOGrady,2,1,20
4,24385,44,155507136,author_id: 155507136,conv_id: 1066662602443538432,1066662602443538048,2018-11-25T11:59:16.000Z,NaN,1066662602443538048,id: 1066662602443538432,...,1,Mark RUTTE,Netherlands,@MinPres,Dutch Government,Prime Minister,MinPres,3,1,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2922,5440,76,3131144855,author_id: 3131144855,conv_id: 587940830511505408,587940830511505408,2015-04-14T11:29:52.000Z,"[{'start': 65, 'end': 79, 'username': 'Conserv...",587940830511505408,id: 587940830511505408,...,1,Boris JOHNSON,England,@BorisJohnson,Foreign Secretary,UK Government,BorisJohnson,0,1,10
2923,5441,77,3131144855,author_id: 3131144855,conv_id: 587918959829241856,587918959829241856,2015-04-14T10:02:57.000Z,"[{'start': 1, 'end': 15, 'username': 'Conserva...",587918959829241856,id: 587918959829241856,...,1,Boris JOHNSON,England,@BorisJohnson,Foreign Secretary,UK Government,BorisJohnson,0,1,10
2924,5466,102,3131144855,author_id: 3131144855,conv_id: 585376346324541440,585376346324541440,2015-04-07T09:39:31.000Z,NaN,585376346324541440,id: 585376346324541440,...,1,Boris JOHNSON,England,@BorisJohnson,Foreign Secretary,UK Government,BorisJohnson,1,1,10
2925,5545,40,747807250819981312,author_id: 747807250819981312,conv_id: 1070664353668587521,1070664353668588032,2018-12-06T13:00:48.000Z,"[{'start': 3, 'end': 19, 'username': '10Downin...",1070664353668588032,id: 1070664353668587521,...,1,Theresa MAY,England,@theresa_may,Prime Minister,UK Government,theresa_may,1,1,10


In [148]:
comparison2 = [
    ['rows_all_data', data.shape[0]],
    ['rows_data_with_words', data2.shape[0]],
    ['Diff', (data.shape[0] - data2.shape[0])]]

print(tabulate(comparison2, tablefmt="grid"))

+----------------------+--------+
| rows_all_data        | 299715 |
+----------------------+--------+
| rows_data_with_words |   2927 |
+----------------------+--------+
| Diff                 | 296788 |
+----------------------+--------+


### English tweets only:

In [149]:
#pip install langid

In [150]:
data.text

0         This no-confidence vote was a chaotic detour. ...
1         Uncertainty is throttling our economy and dest...
2         Today's delay of the Brexit vote is yet anothe...
3         Our latest @CBI_Economics Forecast shows that ...
4         Parity of esteem for vocational qualifications...
                                ...                        
299710    I'm heading to Southampton to meet some of the...
299711    RT @HBaldwinMP: 10 Resolutions To Help You Suc...
299712    In 2014 we must carry on working through our l...
299713    2014 is the year when the whole country can ri...
299714                                      Happy New Year!
Name: text, Length: 299715, dtype: object

In [151]:
import langid
language_classifications = []
tokens = []
for row in data.text:
    language_classification, score = langid.classify(row)
    language_classifications.append(language_classification)

In [152]:
language = pd.DataFrame(language_classifications)
language

,0
0,en
1,en
2,en
3,en
4,en
...,...
299710,en
299711,en
299712,en
299713,en


In [154]:
data['language'] = language

In [155]:
data3 = data[data.language == 'en']
data3

,Unnamed: 0,Unnamed: 0.1,author_id,author_id_new,conv_id_new,conversation_id,created_at,entities.mentions,id,id_new,...,Name,Born_in,twitter_user_name,Role,Place,clean_user_name,score_key_words,score_KOP,total_score,language
0,0,0,4178442737,author_id: 4178442737,conv_id: 1072961418083926023,1072961418083926016,2018-12-12T21:08:31.000Z,NaN,1072961418083926016,id: 1072961418083926023,...,Carolyn FAIRBAIRN,British,@cbicarolyn,Director General,Confederation of British Industry,cbicarolyn,0,1,5,en
1,1,1,4178442737,author_id: 4178442737,conv_id: 1072915641080971264,1072915641080971008,2018-12-12T18:06:37.000Z,NaN,1072915641080971008,id: 1072915641080971264,...,Carolyn FAIRBAIRN,British,@cbicarolyn,Director General,Confederation of British Industry,cbicarolyn,0,1,5,en
2,2,2,4178442737,author_id: 4178442737,conv_id: 1072161702781878274,1072161702781878016,2018-12-10T16:10:44.000Z,NaN,1072161702781878016,id: 1072161702781878274,...,Carolyn FAIRBAIRN,British,@cbicarolyn,Director General,Confederation of British Industry,cbicarolyn,0,1,5,en
3,3,3,4178442737,author_id: 4178442737,conv_id: 1070743996367073280,1070743996367073024,2018-12-06T18:17:16.000Z,"[{'start': 11, 'end': 25, 'username': 'CBI_Eco...",1070743996367073024,id: 1070743996367073280,...,Carolyn FAIRBAIRN,British,@cbicarolyn,Director General,Confederation of British Industry,cbicarolyn,0,1,5,en
4,4,4,4178442737,author_id: 4178442737,conv_id: 1070598700098043906,1070598700098044032,2018-12-06T08:39:55.000Z,NaN,1070598700098044032,id: 1070598700098043906,...,Carolyn FAIRBAIRN,British,@cbicarolyn,Director General,Confederation of British Industry,cbicarolyn,0,1,5,en
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299710,299710,81,103065157,author_id: 103065157,conv_id: 418657196315246592,418657196315246592,2014-01-02T08:17:08.000Z,NaN,418657196315246592,id: 418657196315246592,...,David Cameron,England,@David_Cameron,No-KOP,No-KOP,David_Cameron,0,1,5,en
299711,299711,82,103065157,author_id: 103065157,conv_id: 418432640753340416,418432640753340416,2014-01-01T17:24:50.000Z,"[{'start': 86, 'end': 98, 'username': 'BuzzFee...",418432640753340416,id: 418432640753340416,...,David Cameron,England,@David_Cameron,No-KOP,No-KOP,David_Cameron,0,1,5,en
299712,299712,83,103065157,author_id: 103065157,conv_id: 418350235652337664,418350235652337728,2014-01-01T11:57:23.000Z,NaN,418350235652337728,id: 418350235652337664,...,David Cameron,England,@David_Cameron,No-KOP,No-KOP,David_Cameron,0,1,5,en
299713,299713,84,103065157,author_id: 103065157,conv_id: 418309939707133952,418309939707134016,2014-01-01T09:17:16.000Z,"[{'start': 76, 'end': 85, 'username': 'thetime...",418309939707134016,id: 418309939707133952,...,David Cameron,England,@David_Cameron,No-KOP,No-KOP,David_Cameron,0,1,5,en


In [156]:
comparison3 = [
    ['rows_all_data', data.shape[0]],
    ['rows_data_with_words', data3.shape[0]],
    ['Diff', (data.shape[0] - data3.shape[0])]]

print(tabulate(comparison3, tablefmt="grid"))

+----------------------+--------+
| rows_all_data        | 299715 |
+----------------------+--------+
| rows_data_with_words | 233455 |
+----------------------+--------+
| Diff                 |  66260 |
+----------------------+--------+


### +-3 days:

-----------

### All conditions together:

In [157]:
language_classifications = []
tokens = []
for row in data2.text:
    language_classification, score = langid.classify(row)
    language_classifications.append(language_classification)

In [158]:
language = pd.DataFrame(language_classifications)
data2['language'] = language
data2 #only english tweets with our key_words

,Unnamed: 0,Unnamed: 0.1,author_id,author_id_new,conv_id_new,conversation_id,created_at,entities.mentions,id,id_new,...,Name,Born_in,twitter_user_name,Role,Place,clean_user_name,score_key_words,score_KOP,total_score,language
0,26598,188,2797521996,author_id: 2797521996,conv_id: 740159086667411456,740159086667411456,2016-06-07T12:30:52.000Z,NaN,740159086667411456,id: 740159086667411456,...,David DAVIS,England,@DavidDavisMP,Secretary of State for Exiting the,European Union UK Government,DavidDavisMP,4,1,25,en
1,24792,451,155507136,author_id: 155507136,conv_id: 953958453554401283,953958453554401280,2018-01-18T11:53:13.000Z,"[{'start': 28, 'end': 41, 'username': 'GOettin...",953958453554401280,id: 953958453554401283,...,Mark RUTTE,Netherlands,@MinPres,Dutch Government,Prime Minister,MinPres,2,1,20,nl
2,3894,26,3131144855,author_id: 3131144855,conv_id: 1069285577869205506,1069285577869206016,2018-12-02T17:47:45.000Z,NaN,1069287014867054976,id: 1069287014867054595,...,Boris JOHNSON,England,@BorisJohnson,Foreign Secretary,UK Government,BorisJohnson,3,1,20,en
3,7496,22,540274837,author_id: 540274837,conv_id: 821274533080551424,821274533080551424,2017-01-17T08:39:25.000Z,NaN,821275721310044160,id: 821275721310044160,...,Frances O'GRADY,England,@FrancesOGrady,General Secretary,Trades Union Congress,FrancesOGrady,2,1,20,en
4,24385,44,155507136,author_id: 155507136,conv_id: 1066662602443538432,1066662602443538048,2018-11-25T11:59:16.000Z,NaN,1066662602443538048,id: 1066662602443538432,...,Mark RUTTE,Netherlands,@MinPres,Dutch Government,Prime Minister,MinPres,3,1,20,nl
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2922,5440,76,3131144855,author_id: 3131144855,conv_id: 587940830511505408,587940830511505408,2015-04-14T11:29:52.000Z,"[{'start': 65, 'end': 79, 'username': 'Conserv...",587940830511505408,id: 587940830511505408,...,Boris JOHNSON,England,@BorisJohnson,Foreign Secretary,UK Government,BorisJohnson,0,1,10,en
2923,5441,77,3131144855,author_id: 3131144855,conv_id: 587918959829241856,587918959829241856,2015-04-14T10:02:57.000Z,"[{'start': 1, 'end': 15, 'username': 'Conserva...",587918959829241856,id: 587918959829241856,...,Boris JOHNSON,England,@BorisJohnson,Foreign Secretary,UK Government,BorisJohnson,0,1,10,en
2924,5466,102,3131144855,author_id: 3131144855,conv_id: 585376346324541440,585376346324541440,2015-04-07T09:39:31.000Z,NaN,585376346324541440,id: 585376346324541440,...,Boris JOHNSON,England,@BorisJohnson,Foreign Secretary,UK Government,BorisJohnson,1,1,10,en
2925,5545,40,747807250819981312,author_id: 747807250819981312,conv_id: 1070664353668587521,1070664353668588032,2018-12-06T13:00:48.000Z,"[{'start': 3, 'end': 19, 'username': '10Downin...",1070664353668588032,id: 1070664353668587521,...,Theresa MAY,England,@theresa_may,Prime Minister,UK Government,theresa_may,1,1,10,en


In [159]:
data_with_replys = data2[data2['public_metrics.reply_count'] > 0]
data_with_replys

,Unnamed: 0,Unnamed: 0.1,author_id,author_id_new,conv_id_new,conversation_id,created_at,entities.mentions,id,id_new,...,Name,Born_in,twitter_user_name,Role,Place,clean_user_name,score_key_words,score_KOP,total_score,language
0,26598,188,2797521996,author_id: 2797521996,conv_id: 740159086667411456,740159086667411456,2016-06-07T12:30:52.000Z,NaN,740159086667411456,id: 740159086667411456,...,David DAVIS,England,@DavidDavisMP,Secretary of State for Exiting the,European Union UK Government,DavidDavisMP,4,1,25,en
1,24792,451,155507136,author_id: 155507136,conv_id: 953958453554401283,953958453554401280,2018-01-18T11:53:13.000Z,"[{'start': 28, 'end': 41, 'username': 'GOettin...",953958453554401280,id: 953958453554401283,...,Mark RUTTE,Netherlands,@MinPres,Dutch Government,Prime Minister,MinPres,2,1,20,nl
2,3894,26,3131144855,author_id: 3131144855,conv_id: 1069285577869205506,1069285577869206016,2018-12-02T17:47:45.000Z,NaN,1069287014867054976,id: 1069287014867054595,...,Boris JOHNSON,England,@BorisJohnson,Foreign Secretary,UK Government,BorisJohnson,3,1,20,en
3,7496,22,540274837,author_id: 540274837,conv_id: 821274533080551424,821274533080551424,2017-01-17T08:39:25.000Z,NaN,821275721310044160,id: 821275721310044160,...,Frances O'GRADY,England,@FrancesOGrady,General Secretary,Trades Union Congress,FrancesOGrady,2,1,20,en
4,24385,44,155507136,author_id: 155507136,conv_id: 1066662602443538432,1066662602443538048,2018-11-25T11:59:16.000Z,NaN,1066662602443538048,id: 1066662602443538432,...,Mark RUTTE,Netherlands,@MinPres,Dutch Government,Prime Minister,MinPres,3,1,20,nl
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2921,5439,75,3131144855,author_id: 3131144855,conv_id: 587941282191925248,587941282191925248,2015-04-14T11:31:39.000Z,"[{'start': 74, 'end': 88, 'username': 'Conserv...",587941282191925248,id: 587941282191925248,...,Boris JOHNSON,England,@BorisJohnson,Foreign Secretary,UK Government,BorisJohnson,0,1,10,en
2922,5440,76,3131144855,author_id: 3131144855,conv_id: 587940830511505408,587940830511505408,2015-04-14T11:29:52.000Z,"[{'start': 65, 'end': 79, 'username': 'Conserv...",587940830511505408,id: 587940830511505408,...,Boris JOHNSON,England,@BorisJohnson,Foreign Secretary,UK Government,BorisJohnson,0,1,10,en
2923,5441,77,3131144855,author_id: 3131144855,conv_id: 587918959829241856,587918959829241856,2015-04-14T10:02:57.000Z,"[{'start': 1, 'end': 15, 'username': 'Conserva...",587918959829241856,id: 587918959829241856,...,Boris JOHNSON,England,@BorisJohnson,Foreign Secretary,UK Government,BorisJohnson,0,1,10,en
2924,5466,102,3131144855,author_id: 3131144855,conv_id: 585376346324541440,585376346324541440,2015-04-07T09:39:31.000Z,NaN,585376346324541440,id: 585376346324541440,...,Boris JOHNSON,England,@BorisJohnson,Foreign Secretary,UK Government,BorisJohnson,1,1,10,en


In [160]:
data_with_replys.to_excel('tweets_table_filtered_final.xlsx')

### number of tweets reply pairs:

In [137]:
data_with_replys['public_metrics.reply_count'].sum()

218026

In [135]:
data_with_replys.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'author_id', 'author_id_new',
       'conv_id_new', 'conversation_id', 'created_at', 'entities.mentions',
       'id', 'id_new', 'public_metrics.like_count',
       'public_metrics.quote_count', 'public_metrics.reply_count',
       'public_metrics.retweet_count', 'referenced_tweets', 'text',
       'users.created_at', 'users.description', 'users.id', 'users.name',
       'users.public_metrics.followers_count',
       'users.public_metrics.following_count',
       'users.public_metrics.listed_count', 'users.public_metrics.tweet_count',
       'users.username', 'users.verified', 'text_tokens', 'created_at_date',
       'is_in_special_date', 'Event_Date', 'Event Description',
       'Key opnion leaders of event', 'KOP_num', 'Index', 'Name', 'Born_in',
       'twitter_user_name', 'Role', 'Place', 'clean_user_name',
       'score_key_words', 'score_KOP', 'total_score', 'language'],
      dtype='object')